In [1]:
import torch
from torch import nn
from src.utils.preprocess_utils import notes_to_midi, midi_to_note_indices, one_hot_encode
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [2]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, input_size]
        out, _ = self.lstm(x)  # out: [batch_size, sequence_length, hidden_size]
        out = self.fc(out)
        return out

In [3]:
# Load model
model_name = "2025-01-05_19-47-45_h128_l5_sl60"
model = torch.load(f'weights/model_1/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_14736\40510903.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_1/{model_name}')


LSTMModel(
  (lstm): LSTM(85, 128, num_layers=5, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [4]:
# Load input
input = midi_to_note_indices('data/input.mid', 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [5]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_sequence_one_hot = one_hot_encode(start_sequence, num_notes=num_notes)
    start_tensor = torch.tensor(start_sequence_one_hot, dtype=torch.float32).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(one_hot_encode(generated_sequence_tensor, num_notes=num_notes), dtype=torch.float32).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 23, 14, 3, 28, 16, 7, 21, 9, 53, 55, 53, 51, 6, 52, 4, 3, 55, 48, 51, 6, 54, 51, 6, 6, 52, 22, 4, 48, 3, 28, 31, 52, 34, 16, 4, 58, 55, 4, 67, 67, 52, 65, 67, 66, 64, 65, 64, 55, 73, 69, 73, 61, 63, 72, 73, 59, 68, 59, 55, 80, 63, 63, 55, 53, 65, 63, 55, 57, 54, 5, 17, 5, 5, 33, 48, 18, 6, 50, 51, 48, 44, 48, 51, 6, 48, 6, 50, 51, 51, 6, 48, 1, 4, 16, 4, 7, 35, 50, 6, 34, 3, 30, 8, 6, 35, 3, 21, 6, 45, 6, 8, 42, 17, 35, 7, 24, 6, 21, 6, 8, 8, 21, 42, 33, 8, 36, 38, 17, 5, 8, 8, 11, 21, 8, 6, 45, 6, 6, 16, 4, 7, 50, 13, 35, 40, 4, 16, 4, 7, 19, 4, 7, 34, 3, 3, 24, 6, 6, 33, 8, 83, 6, 24, 6, 6, 22, 4, 7, 48, 50, 6, 33, 20, 8, 21, 9, 8, 32, 8, 5, 35, 36, 35, 6, 3, 4, 38, 24, 6, 21, 6, 45, 6, 8, 5, 21, 8, 5, 48, 8]


In [6]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_1/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_1/2025-01-04_23-42-30_t0.3_w200_M_2025-01-04_23-35-14_h128_l5_sl60.mid
